In [1]:
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from segmentation.scr.utils.utils import set_seed
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

from segmentation.scr.utils import losses, transforms


from segmentation.scr.utils.utils import set_seed
from segmentation.scr.utils.metrics import dice_coef
from segmentation.scr.utils.utils import save_model

from segmentation.models.smp.decoder import Unet

In [46]:
class CFG:
    # ============== pred target =============
    target_size = 1

    # ============== model CFG =============
    model_name = 'Unet'
    backbone = 'mobilenet_v2'

    device = "cuda" if torch.cuda.is_available() else "cpu"
    in_chans = 5 # 65
    # ============== training CFG =============
    image_size = 512
    input_size=512

    valid_batch_size = 4
    lr = 6e-4
    chopping_percentile=1e-3
    # ============== fold =============
    valid_id = 1

    # ============== augmentation =============
   
    valid_aug_list = [
        ToTensorV2(transpose_mask=True),
    ]
    valid_aug = A.Compose(valid_aug_list, p=1.0)

In [6]:
path_img_dir="data\\train\\kidney_3_sparse\\images"
path_lb_dir="data\\train\\kidney_3_dense\\labels"
path_img_dir = Path(path_img_dir)
path_lb_dir = Path(path_lb_dir)

path_img_dir = sorted(list(path_img_dir.rglob("*.tif")))
path_lb_dir = sorted(list(path_lb_dir.rglob("*.tif")))

images_labels = defaultdict(list)
for img in path_img_dir:
    images_labels[img.name].append(img)
for lb in path_lb_dir:
    images_labels[lb.name].append(lb)
new_dict = dict(filter(lambda item: len(item[1]) > 1, images_labels.items()))
print(f"Общее число изображений с масками сегментации : {len(new_dict)}")
img_path = list(map(lambda key : str(new_dict[key][0]) , new_dict))
lb_path = list(map(lambda key : str(new_dict[key][1]), new_dict))

Общее число изображений с масками сегментации : 501


In [7]:
class Data_loader(Dataset):
    def __init__(self,paths,is_label):
        self.paths=paths
        self.paths.sort()
        self.is_label=is_label
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,index):
        img=cv2.imread(self.paths[index],cv2.IMREAD_GRAYSCALE)
        img=torch.from_numpy(img)
        if self.is_label:
            img=(img!=0).to(torch.uint8)*255
        else:
            img=img.to(torch.uint8)
        return img

In [8]:
def resize_to_size( img, image_size = CFG.image_size):
        '''
        resizes the image if its side is smaller than image_size
        padding is the average between the maximum and minimum pixels on the border
        '''
        #print(img.shape)
        if image_size > img.shape[0]:
            #print(1)
            start = ( image_size - img.shape[0])//2
            val1 = (min(img[0,:]).item() + max(img[0,:]).item()) // 2
            val2 = (min(img[img.shape[0] - 1,:]).item() + max(img[img.shape[0] - 1,:]).item()) // 2
            print(val1, val2)
            top= torch.full((start,img.shape[1]),(val1 + val2)//2).to(torch.uint8)
            botton = torch.full((start,img.shape[1]),(val1 + val2)//2).to(torch.uint8)
            #print(border.shape)
            img=torch.cat((top, img, botton), axis = 0).to(torch.uint8)
        if image_size > img.shape[1]:
            #print(2)
            start = ( image_size - img.shape[1])//2
            val1 = (min(img[:, 0]).item() + max(img[:, 0]).item()) // 2
            val2 = (min(img[:, img.shape[1] - 1]).item() + max(img[:, img.shape[1] - 1]).item()) // 2
            left= torch.full((img.shape[0], start),(val1 + val2)//2).to(torch.uint8)
            right = torch.full((img.shape[0], start),(val1 + val2)//2).to(torch.uint8)
            img = torch.cat((left, img, right), axis = 1).to(torch.uint8)
        
        return img

In [9]:
CHOPPING_PER = 1e-3
def min_max_normalization(x:torch.Tensor)->torch.Tensor:
    """input.shape=(batch,f1,...)"""
    shape=x.shape
    if x.ndim>2:
        x=x.reshape(x.shape[0],-1)
    
    min_=x.min(dim=-1,keepdim=True)[0]
    max_=x.max(dim=-1,keepdim=True)[0]
    if min_.mean()==0 and max_.mean()==1:
        return x.reshape(shape)
    
    x=(x-min_)/(max_-min_+1e-9)
    return x.reshape(shape)

def norm_with_clip(x:torch.Tensor,smooth=1e-5):
    dim=list(range(1,x.ndim))
    mean=x.mean(dim=dim,keepdim=True)
    std=x.std(dim=dim,keepdim=True)
    x=(x-mean)/(std+smooth)
    x[x>5]=(x[x>5]-5)*1e-3 +5
    x[x<-3]=(x[x<-3]+3)*1e-3-3
    return x

def filter_noise(x):
    TH=x.reshape(-1)
    index = -int(len(TH) * CFG.chopping_percentile)
    TH:int = np.partition(TH, index)[index]
    x[x>TH]=int(TH)
    ########################################################################
    TH=x.reshape(-1)
    index = -int(len(TH) * CFG.chopping_percentile)
    TH:int = np.partition(TH, -index)[-index]
    x[x<TH]=int(TH)
    return x


In [10]:
def load_data(paths,is_label=False):
    data_loader=Data_loader(paths,is_label)
    data_loader=DataLoader(data_loader, batch_size=16)
    data=[]
    for x in tqdm(data_loader):
        data.append(x)
    x=torch.cat(data,dim=0)
    del data
    if not is_label:
      #  ########################################################################
      x = filter_noise(x)
      x=(min_max_normalization(x.to(torch.float16)[None])[0]*255).to(torch.uint8)
    return x

In [11]:
val_x=load_data(img_path ,is_label=False)
print(val_x.shape)
val_y=load_data(lb_path ,is_label=True)
print(val_y.shape)

100%|██████████| 32/32 [00:07<00:00,  4.22it/s]


torch.Size([501, 1706, 1510])


100%|██████████| 32/32 [00:07<00:00,  4.50it/s]


torch.Size([501, 1706, 1510])


In [48]:
import torch.nn as nn
import segmentation_models_pytorch as smp

class CustomModel(nn.Module):
    def __init__(self, CFG, weight=None):
        super().__init__()
        self.model = smp.Unet(
            encoder_name='mobilenet_v2', 
            encoder_weights=None,
            in_channels=5,
            classes=1,
            activation=None,
        )

    def forward(self, image):
        output = self.model(image)
        # output = output.squeeze(-1)
        return output[:,0]#.sigmoid()


class WrappedModel(nn.Module):
	def __init__(self):
		super(WrappedModel, self).__init__()
		self.module = CustomModel(CFG) 
	def forward(self, x):
		return self.module(x)



In [37]:
class Kaggld_Dataset(Dataset):
    def __init__(self,x:list,y:list,arg=False):
        super(Dataset,self).__init__()
        self.x=x#list[(C,H,W),...]
        self.y=y#list[(C,H,W),...]
        self.image_size=CFG.image_size
        self.in_chans=CFG.in_chans
        self.arg=arg
        if arg:
            self.transform=0
        else: 
            self.transform=CFG.valid_aug
            
    def __len__(self) -> int:
        return sum([y.shape[0]-self.in_chans for y in self.y])
    
    def __getitem__(self,index):
        i=0
        for x in self.x:
            if index>x.shape[0]-self.in_chans:
                index-=x.shape[0]-self.in_chans
                i+=1
            else:
                break
        x=self.x[i]
        y=self.y[i]
        
        x_index=np.random.randint(0,x.shape[1]-self.image_size)
        y_index=np.random.randint(0,x.shape[2]-self.image_size)

        x=x[index:index+self.in_chans,x_index:x_index+self.image_size,y_index:y_index+self.image_size]
        y=y[index+self.in_chans//2,x_index:x_index+self.image_size,y_index:y_index+self.image_size]

        data = self.transform(image=x.numpy().transpose(1,2,0), mask=y.numpy())
        x = data['image']
        y = data['mask']>=127
       
        if self.arg:
            if np.random.randint(2):
                x=x.flip(dims=(0,))
                   
                        
        return x,y#(uint8,uint8)

In [49]:
path_to_model = 'C:\\Users\\123\\Desktop\\resnet baseline\\state_mobnet_kaagle_0t84.pth'

In [50]:
model = WrappedModel()

model = model.cuda()

In [51]:
if not torch.cuda.is_available() or CFG.device == "cpu":
    ckpt = torch.load(path_to_model, map_location=torch.device("cpu"))
elif CFG.device == "cuda" and torch.cuda.is_available():
    ckpt = torch.load(path_to_model)

ckpt = ckpt["model"]

model.load_state_dict(ckpt)

<All keys matched successfully>

In [22]:
def tta(x, model):
    model.eval()
    #x = x.cuda().to(torch.float32)
    #x = norm_with_clip(x.reshape(-1, *x.shape[2:])).reshape(x.shape)
    x_n=[torch.rot90(x,k=i,dims=(-2,-1)) for i in range(4)]
    batch_size = x.shape[0]
    shape = x.shape
    for i in range(4):
        if i ==0:
            x_n[i] = x_n[i]
        else:
            x_n[i]=torch.flip(x_n[i] , dims = (-2,))
    with torch.no_grad():
        pred = [model(b) for b in x_n]
    
        torch.flip(pred[i] , dims = (-2,))
    pred=torch.cat(pred,dim=0)
    pred = pred.sigmoid()
    pred =pred.reshape(4,shape[0],*shape[2:])
    
    
    for i in range(4):
        if i ==0:
            pred[i] = pred[i]
        else:
            pred[i]=torch.flip(pred[i] , dims = (-2,))
    pred=[torch.rot90(pred[i],k=-i,dims=(-2,-1)) for i in range(4)]
    pred=torch.stack(pred,dim=0).mean(0)
            

    return pred
    
    

In [41]:
model = model.cuda()
model.eval()
loss_fn=losses.DiceLoss()
set_seed(42)
val_dataset=Kaggld_Dataset([val_x],[val_y])
val_loader = DataLoader(val_dataset, batch_size=CFG.valid_batch_size, shuffle=False)
timer = tqdm(range(len(val_loader)))
val_losss = 0
val_scores = 0
scores = []
for i, (x, y) in enumerate(val_loader):
    x = x.cuda().to(torch.float32)
    y = y.cuda().to(torch.float32)
    x = norm_with_clip(x.reshape(-1, *x.shape[2:])).reshape(x.shape)

    with torch.no_grad():
        pred = tta(x)
       
    score = dice_coef(pred.detach(), y, from_logits= False)
    scores.append(score.detach().cpu().item())
    val_scores = (val_scores*i+score)/(i+1)
    timer.set_description(
            f"val--> score:{val_scores:.4f}")
    timer.update()
timer.close()


  0%|          | 0/125 [00:31<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
scipy.stats.ttest_rel(a=scores_b, b= scores, alternative='greater')

In [52]:
model.eval()
loss_fn=losses.DiceLoss()
set_seed(42)
val_dataset=Kaggld_Dataset([val_x],[val_y])
val_loader = DataLoader(val_dataset, batch_size=CFG.valid_batch_size, shuffle=False)
timer = tqdm(range(len(val_loader)))
val_losss = 0
val_scores = 0
scores= []
for i, (x, y) in enumerate(val_loader):
    x = x.cuda().to(torch.float32)
    y = y.cuda().to(torch.float32)
    x = norm_with_clip(x.reshape(-1, *x.shape[2:])).reshape(x.shape)

    with torch.no_grad():
        pred = model(x)
        
    score = dice_coef(pred.detach(), y)
    scores.append(score.detach().cpu().item())
    val_scores = (val_scores*i+score)/(i+1)
    timer.set_description(
            f"val-->loss:{val_losss:.4f},score:{val_scores:.4f}")
    timer.update()
timer.close()
print(np.mean(scores))

val-->loss:0.0000,score:0.8228: 100%|██████████| 124/124 [00:07<00:00, 15.56it/s]

0.8228455830004907


In [ ]:
0.8321